In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np; np.set_printoptions(precision=4); np.random.seed(0)
import torch; torch.set_printoptions(precision=4)
seed = 1

torch.manual_seed(seed)
import torch.nn as nn
import matplotlib.pyplot as plt; plt.rc('font', size=12); 
import matplotlib 
from matplotlib.font_manager import FontProperties
from mpl_toolkits import mplot3d
import matplotlib.pylab as pl
import seaborn as sns
import time
import sys
import itertools
import random; random.seed(0)
import scipy
import os
import warnings

from textwrap import wrap
from scipy.stats import wilcoxon
from sklearn.metrics.pairwise import cosine_similarity

sys.path.append("../two_module_rnn/code")
os.chdir('/home/yl4317/Documents/two_module_rnn/code')
from model_working import *
from functions import *
os.chdir('/home/yl4317/Documents/two_module_rnn/')

print(torch.__version__)
print(sys.version)
                
%matplotlib inline

torch.backends.cudnn.benchmark = False
torch.use_deterministic_algorithms(True) 
torch.backends.cudnn.deterministic = True    

# Generate data: performance after silencing SST, VIP or PV cells in the sensorimotor module

In [ ]:
start = time.time()

all_data_opto_perf = []

for model_name in sorted(os.listdir('/scratch/yl4317/two_module_rnn/saved_models/')):
    if ('2023-05-10' in model_name) and 'wcst' in model_name and ('success' in model_name):
        print(model_name)
        path_to_file = '/scratch/yl4317/two_module_rnn/saved_models/' + model_name
        
        with HiddenPrints():
            model, hp_test, hp_task_test, optim, saved_data = load_model_v2(path_to_file=path_to_file, model_name=model_name, simple=False, plot=False, toprint=False)
     
        with open('/scratch/yl4317/two_module_rnn/saved_testdata/{}_testdata_silenceSRSST_noiseless_no_current_matrix'.format(model_name), 'rb') as f: 
            neural_data_silenceSST = pickle.load(f)  
#         test_data_inhibit_sst = generate_neural_data_test(model=model, n_trials_test=100, switch_every_test=10, batch_size=10, to_plot=False, hp_test=hp_test, hp_task_test=hp_task_test, 
#                                               compute_current=False, random_switch=True, n_switches=10, concat_activity=False, opto=opto_params_sst)
        test_data_inhibit_sst = neural_data_silenceSST['test_data']
        mean_perf_nosst = np.mean([_[0] for _ in test_data_inhibit_sst['perfs']])
        print('mean_perf_nosst={}'.format(mean_perf_nosst))
        mean_perf_rule_nosst = np.mean([_[0] for _ in test_data_inhibit_sst['perf_rules']])
        
        with open('/scratch/yl4317/two_module_rnn/saved_testdata/{}_testdata_silenceSRVIP_noiseless_no_current_matrix'.format(model_name), 'rb') as f:
            neural_data_silenceVIP = pickle.load(f)
#         test_data_inhibit_vip = generate_neural_data_test(model=model, n_trials_test=100, switch_every_test=10, batch_size=10, to_plot=False, hp_test=hp_test, hp_task_test=hp_task_test, 
#                                               compute_current=False, random_switch=True, n_switches=10, concat_activity=False, opto=opto_params_vip)
        test_data_inhibit_vip = neural_data_silenceVIP['test_data']
        mean_perf_novip = np.mean([_[0] for _ in test_data_inhibit_vip['perfs']])
        mean_perf_rule_novip = np.mean([_[0] for _ in test_data_inhibit_vip['perf_rules']])
        
        with open('/scratch/yl4317/two_module_rnn/saved_testdata/{}_testdata_silenceSRPV_noiseless_no_current_matrix'.format(model_name), 'rb') as f:
            neural_data_silencePV = pickle.load(f)
#         test_data_inhibit_vip = generate_neural_data_test(model=model, n_trials_test=100, switch_every_test=10, batch_size=10, to_plot=False, hp_test=hp_test, hp_task_test=hp_task_test, 
#                                               compute_current=False, random_switch=True, n_switches=10, concat_activity=False, opto=opto_params_vip)
        test_data_inhibit_pv = neural_data_silencePV['test_data']
        mean_perf_nopv = np.mean([_[0] for _ in test_data_inhibit_pv['perfs']])
        mean_perf_rule_nopv = np.mean([_[0] for _ in test_data_inhibit_pv['perf_rules']])
        
        with open('/scratch/yl4317/two_module_rnn/saved_testdata/{}_testdata_noiseless_no_current_matrix'.format(model_name), 'rb') as f:
            neural_data = pickle.load(f)
        test_data_intact = neural_data['test_data']
        mean_perf_intact = np.mean([_[0] for _ in test_data_intact['perfs']])
        mean_perf_rule_intact = np.mean([_[0] for _ in test_data_intact['perf_rules']])
        if mean_perf_intact<=0.8 or mean_perf_rule_intact<=0.8:
            print('low perf, pass ({}/{})'.format(mean_perf_intact, mean_perf_rule_intact))
            continue
        
        
        all_data_opto_perf.append({'hp': hp_test,
                                 'mean_perf_intact': mean_perf_intact, 
                                 'mean_perf_rule_intact': mean_perf_rule_intact, 
                                 'mean_perf_nosst': mean_perf_nosst,
                                 'mean_perf_novip': mean_perf_novip,
                                 'mean_perf_nopv': mean_perf_nopv,
                                 'mean_perf_rule_nosst': mean_perf_rule_nosst,
                                 'mean_perf_rule_novip': mean_perf_rule_novip,
                                 'mean_perf_rule_nopv': mean_perf_rule_nopv})   
        
# with open('/home/yl4317/Documents/two_module_rnn/processed_data/silencing_perf.pickle', 'wb') as f:
#     pickle.dump(all_data_opto_perf, f)

print(time.time()-start)

# Figure 7e and Supplementary figure 11b: performance after silencing SST

In [ ]:
with open('/home/yl4317/Documents/two_module_rnn/processed_data/silencing_perf.pickle', 'rb') as handle:
    all_data_opto_perf = pickle.load(handle)


data_fig7e = {'intact': [], 'silence_sst': []}
data_suppfig11b = {'intact': [], 'silence_sst': []}

for dend_nonlinear in ['subtractive', 'divisive_2']:
    print(dend_nonlinear)
    fig, ax = plt.subplots(1, 1, figsize=[3.5, 5])
    plt.style.use('classic')
    fig.patch.set_facecolor('white')
    for data in all_data_opto_perf:
        if data['hp']['dend_nonlinearity']!=dend_nonlinear:
            continue
        perf_intact = data['mean_perf_intact']
        perf_nosst = data['mean_perf_nosst']
        ax.plot([perf_intact, perf_nosst], color='k', alpha=0.5, marker='o')
        
        if data['hp']['dend_nonlinearity'] == 'subtractive':
            data_fig7e['intact'].append(perf_intact)
            data_fig7e['silence_sst'].append(perf_nosst)
        elif data['hp']['dend_nonlinearity'] == 'divisive_2':
            data_suppfig11b['intact'].append(perf_intact)
            data_suppfig11b['silence_sst'].append(perf_nosst)

        ax.set_xticks([0, 1])
        ax.set_xticklabels(['Intact', 'Silence SST'], rotation=20)
        ax.set_xlim([-0.5, 1.5])
        ax.set_ylim([0, 1])
        ax.axhline(y=1/3, linestyle='dashed', color='k')
        ax.set_ylabel('Performance', fontsize=20)
        make_pretty_axes(ax)
    fig.tight_layout()
    plt.show()

#     fig.savefig('/home/yl4317/Documents/two_module_rnn/figs/perf_silence_SST_{}.pdf'.format(dend_nonlinear))

# pd.DataFrame.from_dict(data=data_fig7e, orient='columns').to_csv('/home/yl4317/Documents/two_module_rnn/source_data/fig7e_silenceSST_perf_subtractive.csv', header=False)
# pd.DataFrame.from_dict(data=data_suppfig11b, orient='columns').to_csv('/home/yl4317/Documents/two_module_rnn/source_data/suppfig11b_silenceSST_perf_divisive.csv', header=False)


# Supplementary figure 11: performance after silencing PV and VIP in the sensorimotor module

In [ ]:
data_silencepv = {'intact': [], 'silence_pv': []}
data_silencevip = {'intact': [], 'silence_vip': []}


for cell_type in ['pv', 'vip']:
    fig, ax = plt.subplots(figsize=[3.5, 5])
    plt.style.use('classic')
    fig.patch.set_facecolor('white')
    for data in all_data_opto_perf:
        perf_intact = data['mean_perf_intact']
        perf_inactivate = data['mean_perf_no{}'.format(cell_type)]
        ax.plot([perf_intact, perf_inactivate], color='k', alpha=0.5, marker='o')
        if cell_type == 'pv':
            data_silencepv['intact'].append(perf_intact)
            data_silencepv['silence_pv'].append(perf_inactivate)
        elif cell_type == 'vip':
            data_silencevip['intact'].append(perf_intact)
            data_silencevip['silence_vip'].append(perf_inactivate)
        ax.set_xticks([0, 1])
        ax.set_xticklabels(['Intact', 'Silence {}'.format(cell_type)], rotation=20)
        ax.set_xlim([-0.5, 1.5])
        ax.set_ylim([0, 1])
        ax.axhline(y=1/3, linestyle='dashed', color='k')
        ax.set_ylabel('Performance', fontsize=20)
        make_pretty_axes(ax)
        fig.tight_layout()
    plt.show()
#     fig.savefig('/home/yl4317/Documents/two_module_rnn/figs/perf_silence_{}.pdf'.format(cell_type))


# pd.DataFrame.from_dict(data=data_silencepv, orient='columns').to_csv('/home/yl4317/Documents/two_module_rnn/source_data/silencepv.csv', header=False)
# pd.DataFrame.from_dict(data=data_silencevip, orient='columns').to_csv('/home/yl4317/Documents/two_module_rnn/source_data/silencevip.csv', header=False)